#### Installation

* Mysql repository (dpkg) installation

```console
sudo dpkg -i mysql.deb 
&& 
sudo apt install mysql-server mysql-shell
```

#### Setup

```console
sudo nano /etc/mysql/my.cnf
```

```
[mysqld]
port = <port>
bind-address = <ip-addr>
max_allowed_packet=100M
net_buffer_length=1048576
```

```console
systemctl restart mysql
```

#### Startup

```console
mysql
```

* Access denied for the current user

```console
sudo mysql
```

```console
mysql -h <hostname> -P <port> -u <user> -p
```

#### Create an User

```sql
alter user 'root'@'localhost' identified with caching_sha2_password by '<pwd>'
```

* identified with (caching_sha2_password, mysql_native_password)
* some backends require mysql_native_password

```sql
create user '<user>'@'<hostname>' identified with caching_sha2_password by '<pwd>'
``` 

```sql
drop user '<user>'@'<hostname>'
```

```sql
flush privileges
```

#### Give Privileges (User Scopes)

* hostname '%' -> hostname 무시 binding 허용

```sql
show grants for '<user>'@'<hostname>'
```

```sql
grant <privileges> on <database>.<table> to '<user>'@'<hostname>'
```

```sql
grant all privileges on *.* to '<user>'@'<hostname>' with grant option
```

* So-called Read-only Actions

```sql
grant select, show view, process, replication client on *.* to '<any-user>'@'%'
```

* flush privileges is unnecessary

```sql
revoke <privileges> on <database>.<table> from '<user>'@'<hostname>'
```

* List Opened Sessions

```sql
show processlist;
```

#### Set Global Variables

```sql
show global variables like 'local_infile'
```

* local_infile == off

```sql
set global local_infile = 'ON'
```

```sql
show global variables like 'max_allowed_packet'
```

#### Load Database

##### From Local Infile

```console
mysqlsh -u '<user>' -p
```

```javascript
util.loadDump('airport-db',{threads:4, deferTableIndexes:'all', ignoreVersion:true})
```

##### From CSV

```sql
load data infile '<.csv>' into table <table>
  fields terminated by ','
  enclosed by ''
  lines terminated by '\n'
  ignore 1 lines
```

##### From Something Tabular

```python
from sqlalchemy import create_engine
import pandas as pd
import dask.dataframe as df

db_uri='mysql://root:<pwd>@localhost:3306/sample'

df.from_pandas(pd.read_feather('~/res/_gps-review-scrap.feather')).to_sql(
  name='gps',
  conn=create_engine(db_uri)
)
```

#### Get Info

```sql
show databases
```

```sql
show tables from <database>
```

```sql
show columns from <table>
```


#### Clauses

##### Unique Value by Column

```sql
select distinct q,w,e from r
where q + w + e > const;
```

```sql
select q,w,e from r
where q + w + e > const
group by q, w, e;
```

* distinct은 전형적인 unique 메서드임
* 전자는 q, w, e 각각의, 후자는 (q, w, e)에 대해 unique array을 냄

##### Isna-Isblank Equivalent

```sql
select <product_name>, 
  <product_price> * (<product_quant> + ifnull[coalesce](<product_quant_coming>, <fillna_value>)) from product
```

##### Count Rows

```sql
select count(*) as nrows from <table0> union all
  select count(*) from <table1> union all
  select count(*) from <table2>
```

##### Row Insertion to a Result

```sql
select c, count(c) as c_cnt from sumstat 
  where p <= 0.001 group by c union all 
select 'total' as c, count(c) from sumstat 
  where p <= 0.001;
```

##### Group By Aggfunc

```sql
select <column.categorical>,
  avg(datediff(flight_start,flight_end)) as flight_avg,
  count(*) as flight_count from <table>
  where <table0.id> = <table1.id> and <cond>
```

##### Partial String Match with Subquery

```sql
select * from (
  select lower(<table.column>) from <table>
) as lowered
  where <cond> and <table.column> like 'ec%';
```

* Character match _, remainder match %
* '%[o-v]' or set '%[abcdef]' 등 레젝스 유사표현도 like로 사용 가능

##### Boolean Indexing in Subquery Not Being Alias

```sql
select <column.x> from <table.a>
where exists (
  select <column.y> from <table.b>
  where <table.b.column.q> = <table.a.column.w> and <column.z> <cond>
);
```

```sql
select <product_name> from <product>
  where <product_id> = any(
    select <product_id> from <product_order> where <product_quant> = 1
  )
```

* The below shows a boolean indexing resembles to the aboves

```sql
select ...
where exists (
  select 1 from ... where <subquery_where> and <outer_expr> = <inner_expr>
)
```

##### Uint and Zerofill

* SAS식 decimal[numeric](p: precision, d: number of digits) [zerofill: padding, unsigned]

```sql
decimal(5,3)
```

* string casting unsigned를 씀

```sql
select cast(substr(<table.pos>, 1, 2) as unsigned) from <table>
```


##### Aggfunc

* Select로 truth test 가능

```sql
select sum(cast(substring_index(id,':',2) as unsigned) = c) from sumstat where p => 0.001;
```
